In [6]:
import get_price.get_futures_price as gfp
data_out = gfp.get_futures_price_preloaded(ticker_head='EC')


contract_data = data_out[data_out['cont_indx']==201503]

In [13]:
import pandas as pd

contract_data['ema15'] = pd.ewma(contract_data['close_price'], span=15)
contract_data['ema45'] = pd.ewma(contract_data['close_price'], span=45)

contract_data['ppo'] = (contract_data['ema15']-contract_data['ema45'])/contract_data['ema45']

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde